In [43]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
sys.path.append(r'C:\\Users\\scanimage\\Documents\\JJM\\post_cnmfe_analysis')
#sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import os
os.chdir(r'C:\\Users\\scanimage\\Documents\\JJM\\post_cnmfe_analysis')
import python_utils_jjm as utils_jjm
import python_utils_jjm as utils_jjm
import dlc_utils
from sklearn.preprocessing import MinMaxScaler
import scipy.spatial.distance as dist
import itertools
import math
import warnings
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
#import av
from multiprocessing import Pool
import functools
import glob
#plt.rcParams['animation.ffmpeg_path']='/home/jma819/.conda/envs/caiman/bin/ffmpeg'
warnings.filterwarnings(action='once')

C:\Users\scanimage\AppData\Local\Temp\ipykernel_12480\152334368.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [44]:
## list *out.mat files to analyze

In [45]:
dir_path = r'F:\\JJM\\miniscope_analysis\\mGluR5_NAM\\cnmfe_output_files\\'
file_name = '19-Dec_17_39_23_out.mat'
CNMFE_file = dir_path+file_name

## load cnmfe output and perform some basic adjustments (normalize traces to peak intensity, calculate z score)
cell_fluorescence = sio.loadmat(CNMFE_file)
C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)

spatial_components=np.array(cell_fluorescence['A'].todense())

##load spatial components by session
# for v4 dimensions are 600x600 pixels
com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6, dims=(600, 600))
cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components, dims=(600, 600))

# cell_fluorescence['P'][0][0][13][0][0][0].split('/')

print(cell_fluorescence['P'][0][0][13][0][0][0].split('/')[5]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[6]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[9])

grouped_raw_data = {}

session = cell_fluorescence['P'][0][0][13][0][0][0].split('/')[5]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[6]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[9]
grouped_raw_data[session] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 
                               'C_normalized_z_scored': C_normalized_z_scored, 'com' : com_df, 
                               'spatial_components' : spatial_components, 
                               'cell_contours': cell_contours, 'for_dims' : for_dims}

## for spatial clustering start by using z scored data on whole trace as input to event detection function, then try with z score based 
#on rest or other periods
C_traces_filtered_for_analysis = {}
com_filtered_for_analysis = {}
##remove small cells 
for session in list(grouped_raw_data.keys()):
    # get indicies of small cells 
    cells_to_drop = np.array([int(cell) for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    C_traces_filtered = grouped_raw_data[session]['C_normalized_z_scored'].drop(cells_to_drop, axis=1)
    
    C_traces_filtered_for_analysis[session] = C_traces_filtered.drop('msCamFrame', axis=1)
    com_filtered_for_analysis[session] = grouped_raw_data[session]['com'].drop(cells_to_drop, axis=0).transpose()

print('cell count:')
print(np.shape(C_traces_filtered)[1])

# sets sampling interval to 5Hz to correspond to behavior data 
new_sampling_interval = .2
C_z_score_df = pd.concat([C_traces_filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(C_traces_filtered_for_analysis.keys())], axis=1, keys=list(C_traces_filtered_for_analysis.keys()))

# after downsampling convert back to regular index, timedelta is behaving oddly 
C_z_score_df_reindexed = pd.concat([C_z_score_df[session].reset_index(drop=True) 
                                    for session in list(set(C_z_score_df.columns.get_level_values(0)))], axis=1, keys=list(set(C_z_score_df.columns.get_level_values(0))))
# drop columns with all nan values
C_z_score_df_reindexed.dropna(axis=1, how='all', inplace=True)

print(session)

## save each session csv file individually for loading into MATLAB
# using MATLAB readtable function, which will create variable names from column header 
save_directory = r'F:\JJM\miniscope_analysis\mGluR5_NAM\cnmfe_output_files\\'
for session in list(set(C_z_score_df_reindexed.columns.get_level_values(0))):
    C_z_score_df_reindexed[session].to_csv(save_directory+session+'_C_traces_filtered.csv')
    com_filtered_for_analysis[session].to_csv(save_directory+session+'_com_filtered.csv')

print('finished, saved:')
list(set(C_z_score_df_reindexed.columns.get_level_values(0)))

Vehicle_DIO_r2.19_My_V4_Miniscope
cell count:
4
Vehicle_DIO_r2.19_My_V4_Miniscope
finished, saved:


['Vehicle_DIO_r2.19_My_V4_Miniscope']

In [42]:
session

'Vehicle_DIO_r2.19_My_V4_Miniscope'

In [32]:
com_filtered_for_analysis['Fenobam_DIO_r2.19_16_34_31']

,2,5,6,9,12
y,348.084262,205.229577,319.792048,458.16037,220.177264
x,275.628046,309.353020,268.416982,403.26302,437.152203


In [33]:
C_normalized_z_scored

,1,2,3,4,5,6,7,8,9,10,11,12,msCamFrame
0 days 00:00:00,-0.868245,-0.932268,0.042173,-0.230422,-0.970214,-1.059509,-0.739175,-0.695371,0.101818,-0.529828,-0.590118,-1.249263,-1.732002
0 days 00:00:00.050000,-0.868245,-0.932268,-0.024898,-0.280606,-0.970214,-1.059509,-0.739175,-0.695371,0.008230,-0.530383,-0.590118,-1.249263,-1.731905
0 days 00:00:00.100000,-0.868245,-0.932268,-0.087114,-0.324595,-0.970214,-1.059509,-0.739175,-0.695371,-0.078172,-0.530828,-0.590118,-1.249263,-1.731807
0 days 00:00:00.150000,-0.868245,-0.932268,-0.144827,-0.363154,-0.970214,-1.059509,-0.739175,-0.695371,-0.157940,-0.531184,-0.590118,-1.249263,-1.731710
0 days 00:00:00.200000,-0.868245,-0.932268,-0.198363,-0.396953,-0.970214,-1.059509,-0.739175,-0.695371,-0.231583,-0.531469,1.052557,-1.249263,-1.731613
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:29:38.450000,-0.160680,-0.847511,-0.882451,-0.606444,-0.968800,-1.059509,-0.522061,-0.695371,-1.114927,-0.532620,-0.590118,-1.249185,1.731613
0 days 00:29:38.500000,-0.194007,-0.878724,-0.882678,-0.611471,-0.969631,-1.059509,-0.543392,-0.695371,-1.115196,-0.532620,-0.590118,-1.249205,1.731710
0 days 00:29:38.550000,-0.225764,-0.898443,-0.882880,-0.615670,-0.969973,-1.059509,-0.562627,-0.695371,-1.115430,-0.532620,-0.590118,-1.249220,1.731807
0 days 00:29:38.600000,-0.256026,-0.910900,-0.883059,-0.025627,-0.970115,-1.059509,-0.579972,-0.695371,-0.551054,-0.532620,-0.590118,-0.110863,1.731905


In [27]:
C_traces_filtered_for_analysis['Fenobam_DIO_r2.19_16_04_27']

,2,4,10,12,19,20
0 days 00:00:00,1.185376,-0.815337,0.806257,-0.825337,0.030414,-0.720831
0 days 00:00:00.050000,0.901933,-0.821520,0.623493,-0.825337,-0.059252,-0.735261
0 days 00:00:00.100000,0.658137,-0.827136,1.186156,-0.825337,-0.140133,-0.748163
0 days 00:00:00.150000,1.214017,-0.712940,0.967216,-0.825337,-0.213090,-0.323326
0 days 00:00:00.200000,0.926567,-0.728503,1.357565,-0.825337,-0.278899,0.178984
...,...,...,...,...,...,...
0 days 00:29:38.450000,-0.808417,-0.798572,-1.112804,-0.825336,-0.884806,-0.852286
0 days 00:29:38.500000,-0.360822,-0.810260,-1.112876,-0.825337,-0.884806,-0.853086
0 days 00:29:38.550000,-0.536722,-0.820327,-1.112930,-0.825337,-0.884806,-0.853752
0 days 00:29:38.600000,-0.648183,-0.828998,-1.088346,-0.825337,-0.884806,-0.854307


In [28]:
C_z_score_df_reindexed['Fenobam_DIO_r2.19_16_04_27']

,2,4,10,12,19,20
0,1.214017,-0.712940,1.186156,-0.825337,0.030414,-0.323326
1,2.942199,-0.728503,1.357565,-0.825337,-0.278899,0.178984
2,2.413013,-0.777747,1.928697,-0.825337,-0.483675,0.462361
3,1.934985,-0.732263,2.269336,-0.825337,-0.619244,0.464814
4,1.212703,-0.331992,1.840637,-0.825337,-0.708995,0.087693
...,...,...,...,...,...,...
8889,-0.622339,-0.879820,-1.100305,-0.070853,-0.884805,-0.805510
8890,-0.702434,-0.881173,-1.109130,-0.797474,-0.884805,-0.832260
8891,-0.639037,-0.816663,-1.111862,-0.824995,-0.884806,-0.845132
8892,-0.360822,-0.798572,-1.112708,-0.825333,-0.884806,-0.851326


In [68]:
## batch compiling
dir_path = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
file_names = []

/home/jma819/.conda/envs/caiman_with_tables/lib/python3.8/site-packages/jupyter_client/jsonutil.py:64: DeprecationWarning: Interpreting naive datetime as local 2022-11-21 11:59:03.319321. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [4]:
key = {}
for file_name in file_names:
    CNMFE_file = dir_path+file_name
    ## load cnmfe output and perform some basic adjustments (normalize traces to peak intensity, calculate z score)
    cell_fluorescence = sio.loadmat(CNMFE_file)
    C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
    C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
    C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
    C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
    spatial_components=np.array(cell_fluorescence['A'].todense())
    ##load spatial components by session
    # for v4 dimensions are 600x600 pixels
    com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6, dims=(600, 600))
    cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components, dims=(600, 600))
    # cell_fluorescence['P'][0][0][13][0][0][0].split('/')
    print(cell_fluorescence['P'][0][0][13][0][0][0].split('/')[5]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[6]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[8])
    grouped_raw_data = {}
    session = cell_fluorescence['P'][0][0][13][0][0][0].split('/')[5]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[6]+'_'+cell_fluorescence['P'][0][0][13][0][0][0].split('/')[8]
    grouped_raw_data[session] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 
                               'C_normalized_z_scored': C_normalized_z_scored, 'com' : com_df, 
                               'spatial_components' : spatial_components, 
                               'cell_contours': cell_contours, 'for_dims' : for_dims}
    ## for spatial clustering start by using z scored data on whole trace as input to event detection function, then try with z score based 
    #on rest or other periods
    C_traces_filtered_for_analysis = {}
    com_filtered_for_analysis = {}
    ##remove small cells 
    for session in list(grouped_raw_data.keys()):
        # get indicies of small cells 
        cells_to_drop = np.array([int(cell) for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
        C_traces_filtered = grouped_raw_data[session]['C_normalized_z_scored'].drop(cells_to_drop, axis=1)
    
        C_traces_filtered_for_analysis[session] = C_traces_filtered.drop('msCamFrame', axis=1)
        com_filtered_for_analysis[session] = grouped_raw_data[session]['com'].drop(cells_to_drop, axis=0).transpose()
    print('cell count:')
    print(np.shape(C_traces_filtered)[1])
    # sets sampling interval to 5Hz to correspond to behavior data 
    new_sampling_interval = .2
    C_z_score_df = pd.concat([C_traces_filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(C_traces_filtered_for_analysis.keys())], axis=1, keys=list(C_traces_filtered_for_analysis.keys()))
    # after downsampling convert back to regular index, timedelta is behaving oddly 
    C_z_score_df_reindexed = pd.concat([C_z_score_df[session].reset_index(drop=True) 
                                    for session in list(set(C_z_score_df.columns.get_level_values(0)))], axis=1, keys=list(set(C_z_score_df.columns.get_level_values(0))))
    # drop columns with all nan values
    C_z_score_df_reindexed.dropna(axis=1, how='all', inplace=True)
    print(session)
    ## save each session csv file individually for loading into MATLAB
    # using MATLAB readtable function, which will create variable names from column header 
    save_directory = '/projects/b1118/miniscope/analysis/spatial_data/spatial_clustering/jones_script_analysis/'
    for session in list(set(C_z_score_df_reindexed.columns.get_level_values(0))):
        C_z_score_df_reindexed[session].to_csv(save_directory+session+'_C_traces_filtered.csv')
        com_filtered_for_analysis[session].to_csv(save_directory+session+'_com_filtered.csv')
    print('finished, saved:')
    list(set(C_z_score_df_reindexed.columns.get_level_values(0)))
    key[session]=file_name

/home/jma819/.conda/envs/caiman_with_tables/lib/python3.8/site-packages/jupyter_client/jsonutil.py:64: DeprecationWarning: Interpreting naive datetime as local 2023-02-06 14:06:58.394586. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


NameError: name 'file_names' is not defined